# "지하철 이용승객 분석" EDA 프로젝트(1)
- 이번 프로젝트에서는 EDA가 무엇이고 , 실습을 통해 EDA 하는 방법을 배워보겠습니다.
- EDA(Exploratory Data Analysis 탐색적 데이터 분석)는 데이터를 이해하는 과정이며 
- 통계적 요약, 분포 파악 및 시각화 등의 기법을 통해 직관적으로 데이터 특성 파악 할수 있습니다.
- 결국, EDA를 통해 데이터에서 트랜드 분석, 인사이트 도출하게 됩니다. 


## 실습 데이터 
- 지하철 승하자 이용객 데이터 : 2019.01.01 ~ 2019.06.30 
  - 서울열린데이터광장 :  http://data.seoul.go.kr/dataList/OA-12914/A/1/datasetView.do
- 지하철 노선 정보 데이터

## EDA 실습을 통해 해결해야 할 질문들
 
- 2019.01~06 중에 언제 지하철을 가장 많이 이용했을까? (기준: 승하자총승객수)
- 1월~6월중에 5월에 지하철 승객수가 많다? (기준 :승하자총승객수)
- 요일중에서 목요일에 지하철 승객수가 많다? (기준 :승하자총승객수)
- 일자별(월일별) 승하차승객수 그래프 그려 볼까요? (기준 :승하자총승객수)
- 가장 승객이 많이 타는 승차역은?
- 노선별로 역별/요일별 승차승객수를 비교해 볼수 있을까? (1~9호선, 역별/요일별 heatmap)
- 1호선에서 가장 하자를 많이 하는 역은? (groupby)
- 2호선중에서 어느 역에서 승차가 가장 많이 발생할까? (Folium 역 표시)

## 해당 Jupyter Notebook에서 실습 내용
- 필요한 라이브러리 가져오기
- 필요한 파일 불러오고 병합하기(read_csv, merge)
- Feature Engineering : 요일, 연월, 월일, 승하자총승객수 컬럼 추가
- 데이터 저장하기

<br>

# 1. 필요한 라이브러리 가져오기

In [1]:
# 필요한 라이브러리를 불러옵니다.

import pandas as pd


# 2 raw 파일 불러오기
- 열차/노선/지하철역별 승하자 고객수
- 19년 상반기(19년 1월 ~ 19년 6월)

In [2]:
# 작업할 여러개의 파일 중 하나의 파일을 불러옵니다.

file = r'.\rawfiles\CARD_SUBWAY_MONTH_201901.csv'

pd.read_csv(file)


,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20190101,2호선,을지로4가,3862.0,3728.0,20190104
1,20190101,3호선,을지로3가,8104.0,7554.0,20190104
2,20190101,2호선,을지로입구,22478.0,21330.0,20190104
3,20190101,2호선,시청,8381.0,6049.0,20190104
4,20190101,6호선,동묘앞,8045.0,8504.0,20190104
...,...,...,...,...,...,...
18329,20190131,우이신설경전철,북한산보국문,6195.0,5786.0,20190203
18330,20190131,우이신설경전철,정릉,4600.0,4211.0,20190203
18331,20190131,4호선,성신여대입구,3922.0,4289.0,20190203
18332,20190131,6호선,보문,1659.0,1693.0,20190203


In [3]:
raw = pd.read_csv(file)
raw.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20190101,2호선,을지로4가,3862.0,3728.0,20190104
1,20190101,3호선,을지로3가,8104.0,7554.0,20190104
2,20190101,2호선,을지로입구,22478.0,21330.0,20190104
3,20190101,2호선,시청,8381.0,6049.0,20190104
4,20190101,6호선,동묘앞,8045.0,8504.0,20190104


In [4]:
# info() 이용해 데이터 구조를 살펴볼수 있습니다.

raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18334 entries, 0 to 18333
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   사용일자    18334 non-null  int64  
 1   노선명     18334 non-null  object 
 2   역명      18334 non-null  object 
 3   승차총승객수  18334 non-null  float64
 4   하차총승객수  18334 non-null  float64
 5   등록일자    18334 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 859.5+ KB


## 참고) 판다스로 파일 읽어오기

### 데이터 파일 읽기 : read_excel / read_csv
- pd.read_excel('파일경로+파일명.xlsx')
- pd.read_csv('파일경로+파일명.csv', encoding='utf-8')
- pd.read_csv('파일경로+파일명.csv', encoding='cp949') # MS 엑셀에 저장된 경우

In [5]:
# 이번에 여러 파일을 불러오겠습니다.
# 2개 파일 불러와서 합치기

raw = pd.DataFrame()
raw.head()

file1 = r'.\rawfiles\CARD_SUBWAY_MONTH_201901.csv'
file2 = r'.\rawfiles\CARD_SUBWAY_MONTH_201902.csv'

temp1 = pd.read_csv(file1)
temp2 = pd.read_csv(file2)

# pandas concat 합치때, 인덱스 번호 그래로 간다. 옵션 ignore_index=True 사용하면 인덱스 새로 생성
raw = pd.concat([temp1, temp2], axis=0)


In [6]:
raw

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20190101,2호선,을지로4가,3862.0,3728.0,20190104
1,20190101,3호선,을지로3가,8104.0,7554.0,20190104
2,20190101,2호선,을지로입구,22478.0,21330.0,20190104
3,20190101,2호선,시청,8381.0,6049.0,20190104
4,20190101,6호선,동묘앞,8045.0,8504.0,20190104
...,...,...,...,...,...,...
16538,20190228,2호선,을지로4가,6128.0,7087.0,20190303
16539,20190228,2호선,동대문역사문화공원,3387.0,4061.0,20190303
16540,20190228,5호선,청구,4920.0,4474.0,20190303
16541,20190228,1호선,가산디지털단지,51729.0,52043.0,20190303


---

# Q) 폴더에 있는 모든 파일을 불러와서 병합할수 있을까?

In [7]:
# 폴더/파일을 관리하는 os 라이브러리를 불러옵니다.

import os


In [8]:
# os.listdir()을 이용해 폴더에 있는 파일 리스트를 살펴봅니다.

os.listdir()


['1.지하철 이용승객 분석 EDA 프로젝트(1).ipynb', 'myEDA.ipynb', 'rawfiles', 'rawfiles.zip']

In [9]:
dirpath = 'rawfiles'
os.listdir(dirpath)

['CARD_SUBWAY_MONTH_201901.csv',
 'CARD_SUBWAY_MONTH_201902.csv',
 'CARD_SUBWAY_MONTH_201903.csv',
 'CARD_SUBWAY_MONTH_201904.csv',
 'CARD_SUBWAY_MONTH_201905.csv',
 'CARD_SUBWAY_MONTH_201906.csv',
 '지하철노선위경도정보2.csv']

In [10]:
dirpath = './rawfiles/'
files = os.listdir(dirpath)
files


['CARD_SUBWAY_MONTH_201901.csv',
 'CARD_SUBWAY_MONTH_201902.csv',
 'CARD_SUBWAY_MONTH_201903.csv',
 'CARD_SUBWAY_MONTH_201904.csv',
 'CARD_SUBWAY_MONTH_201905.csv',
 'CARD_SUBWAY_MONTH_201906.csv',
 '지하철노선위경도정보2.csv']

In [11]:
# 여러개의 파일을 병합하겠습니다.

# 빈 데이터프레임을 준비
raw = pd.DataFrame()

# 반복문 -> csv 파일 읽기 -> 데이터프레임에 추가하기
for file in files:
    #print(dirpath + file)
    temp = pd.read_csv(dirpath + file)
    raw = pd.concat([raw, temp], axis=0)

raw.tail()


,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,역이름,역지역,위도,경도,호선
970,NaN,NaN,NaN,NaN,NaN,NaN,현충원,대전,36.359070,127.321793,1호선
971,NaN,NaN,NaN,NaN,NaN,NaN,월드컵경기장,대전,36.366849,127.317836,1호선
972,NaN,NaN,NaN,NaN,NaN,NaN,노은,대전,36.374130,127.317866,1호선
973,NaN,NaN,NaN,NaN,NaN,NaN,지족,대전,36.384093,127.319489,1호선
974,NaN,NaN,NaN,NaN,NaN,NaN,반석,대전,36.392253,127.314506,1호선


In [35]:
raw

,사용일자,요일,노선명,역명,승차총승객수,하차총승객수,승하자총승객수,연월,월일,등록일자
0,2019-01-01,월,2호선,을지로4가,3862.0,3728.0,7590.0,2019-01,01-01,20190104.0
1,2019-01-01,월,3호선,을지로3가,8104.0,7554.0,15658.0,2019-01,01-01,20190104.0
2,2019-01-01,월,2호선,을지로입구,22478.0,21330.0,43808.0,2019-01,01-01,20190104.0
3,2019-01-01,월,2호선,시청,8381.0,6049.0,14430.0,2019-01,01-01,20190104.0
4,2019-01-01,월,6호선,동묘앞,8045.0,8504.0,16549.0,2019-01,01-01,20190104.0
...,...,...,...,...,...,...,...,...,...,...
107998,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107999,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108000,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108001,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
raw = raw.reset_index(drop=True)

In [14]:
raw.head(7)

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,역이름,역지역,위도,경도,호선
0,20190101.0,2호선,을지로4가,3862.0,3728.0,20190104.0,NaN,NaN,NaN,NaN,NaN
1,20190101.0,3호선,을지로3가,8104.0,7554.0,20190104.0,NaN,NaN,NaN,NaN,NaN
2,20190101.0,2호선,을지로입구,22478.0,21330.0,20190104.0,NaN,NaN,NaN,NaN,NaN
3,20190101.0,2호선,시청,8381.0,6049.0,20190104.0,NaN,NaN,NaN,NaN,NaN
4,20190101.0,6호선,동묘앞,8045.0,8504.0,20190104.0,NaN,NaN,NaN,NaN,NaN
5,20190101.0,경의선,청량리,15007.0,15397.0,20190104.0,NaN,NaN,NaN,NaN,NaN
6,20190101.0,1호선,제기동,10187.0,10178.0,20190104.0,NaN,NaN,NaN,NaN,NaN


# Q) 요일을 추가할수 있을까?

In [15]:
# 사용일자 컬럼 타입 확인 : int64

raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108003 entries, 0 to 108002
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   사용일자    107028 non-null  float64
 1   노선명     107028 non-null  object 
 2   역명      107028 non-null  object 
 3   승차총승객수  107028 non-null  float64
 4   하차총승객수  107028 non-null  float64
 5   등록일자    107028 non-null  float64
 6   역이름     975 non-null     object 
 7   역지역     975 non-null     object 
 8   위도      975 non-null     float64
 9   경도      975 non-null     float64
 10  호선      975 non-null     object 
dtypes: float64(6), object(5)
memory usage: 9.1+ MB


In [16]:
# '사용일자' 타입 int64 --> DateTime 타입으로 변경 : pandas.to_datetime 사용

raw['사용일자'] = pd.to_datetime(raw['사용일자'], format='%Y%m%d')
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108003 entries, 0 to 108002
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   사용일자    107028 non-null  datetime64[ns]
 1   노선명     107028 non-null  object        
 2   역명      107028 non-null  object        
 3   승차총승객수  107028 non-null  float64       
 4   하차총승객수  107028 non-null  float64       
 5   등록일자    107028 non-null  float64       
 6   역이름     975 non-null     object        
 7   역지역     975 non-null     object        
 8   위도      975 non-null     float64       
 9   경도      975 non-null     float64       
 10  호선      975 non-null     object        
dtypes: datetime64[ns](1), float64(5), object(5)
memory usage: 9.1+ MB


In [17]:
raw.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,역이름,역지역,위도,경도,호선
0,2019-01-01,2호선,을지로4가,3862.0,3728.0,20190104.0,NaN,NaN,NaN,NaN,NaN
1,2019-01-01,3호선,을지로3가,8104.0,7554.0,20190104.0,NaN,NaN,NaN,NaN,NaN
2,2019-01-01,2호선,을지로입구,22478.0,21330.0,20190104.0,NaN,NaN,NaN,NaN,NaN
3,2019-01-01,2호선,시청,8381.0,6049.0,20190104.0,NaN,NaN,NaN,NaN,NaN
4,2019-01-01,6호선,동묘앞,8045.0,8504.0,20190104.0,NaN,NaN,NaN,NaN,NaN


### pandas.Series.dt
- 참고 사이트 : https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.dayofweek.html
  - pandas.Series.dt.date
  - pandas.Series.dt.time
  - pandas.Series.dt.year
  - pandas.Series.dt.month
  - pandas.Series.dt.day

In [18]:
raw['사용일자'].dt.date.head(3)


0    2019-01-01
1    2019-01-01
2    2019-01-01
Name: 사용일자, dtype: object

In [19]:
raw['사용일자'].dt.time.head(3)

0    00:00:00
1    00:00:00
2    00:00:00
Name: 사용일자, dtype: object

In [20]:
raw['사용일자'].dt.year.head(3)

0    2019.0
1    2019.0
2    2019.0
Name: 사용일자, dtype: float64

In [21]:
raw['사용일자'].dt.month.head(3)

0    1.0
1    1.0
2    1.0
Name: 사용일자, dtype: float64

In [22]:
raw['사용일자'].dt.day.head(3)

0    1.0
1    1.0
2    1.0
Name: 사용일자, dtype: float64

In [23]:
# Monday=0, Sunday=6
raw['사용일자'].dt.dayofweek.head(3)

0    1.0
1    1.0
2    1.0
Name: 사용일자, dtype: float64

In [36]:
# 사용일자에 대해서 요일로 변환
raw['요일'] = raw['사용일자'].dt.dayofweek
raw

,사용일자,요일,노선명,역명,승차총승객수,하차총승객수,승하자총승객수,연월,월일,등록일자
0,2019-01-01,1.0,2호선,을지로4가,3862.0,3728.0,7590.0,2019-01,01-01,20190104.0
1,2019-01-01,1.0,3호선,을지로3가,8104.0,7554.0,15658.0,2019-01,01-01,20190104.0
2,2019-01-01,1.0,2호선,을지로입구,22478.0,21330.0,43808.0,2019-01,01-01,20190104.0
3,2019-01-01,1.0,2호선,시청,8381.0,6049.0,14430.0,2019-01,01-01,20190104.0
4,2019-01-01,1.0,6호선,동묘앞,8045.0,8504.0,16549.0,2019-01,01-01,20190104.0
...,...,...,...,...,...,...,...,...,...,...
107998,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107999,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108000,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108001,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
conv = {
    0 : '일',
    1 : '월',
    2 : '화',
    3 : '수',
    4 : '목',
    5 : '금',
    6 : '토'
}

In [26]:
# 숫자 요일을 한글명 요일로 변경
raw['요일'] = raw['요일'].map(conv)


In [27]:
raw.sample(5)

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,역이름,역지역,위도,경도,호선,요일
16384,2019-01-28,5호선,거여,8187.0,7710.0,20190131.0,NaN,NaN,NaN,NaN,NaN,일
94635,2019-06-10,9호선,신반포,3449.0,3350.0,20190613.0,NaN,NaN,NaN,NaN,NaN,일
105970,2019-06-29,1호선,용산,47083.0,51208.0,20190702.0,NaN,NaN,NaN,NaN,NaN,금
73367,2019-05-05,2호선,선릉,15676.0,13288.0,20190508.0,NaN,NaN,NaN,NaN,NaN,토
63321,2019-04-18,2호선,영등포구청,27454.0,27840.0,20190421.0,NaN,NaN,NaN,NaN,NaN,수


# Q) '연월' 컬럼을 만들어 보자!

In [28]:
# pandas.Series.dt.strftime 활용
raw['사용일자'].dt.strftime('%Y-%m')

0         2019-01
1         2019-01
2         2019-01
3         2019-01
4         2019-01
           ...   
107998        NaN
107999        NaN
108000        NaN
108001        NaN
108002        NaN
Name: 사용일자, Length: 108003, dtype: object

In [29]:
# 사용일자 컬럼에서 년월 값을 가져와 년월 컬럼 만들자

raw['연월'] = raw['사용일자'].dt.strftime('%Y-%m')
raw.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,역이름,역지역,위도,경도,호선,요일,연월
0,2019-01-01,2호선,을지로4가,3862.0,3728.0,20190104.0,NaN,NaN,NaN,NaN,NaN,월,2019-01
1,2019-01-01,3호선,을지로3가,8104.0,7554.0,20190104.0,NaN,NaN,NaN,NaN,NaN,월,2019-01
2,2019-01-01,2호선,을지로입구,22478.0,21330.0,20190104.0,NaN,NaN,NaN,NaN,NaN,월,2019-01
3,2019-01-01,2호선,시청,8381.0,6049.0,20190104.0,NaN,NaN,NaN,NaN,NaN,월,2019-01
4,2019-01-01,6호선,동묘앞,8045.0,8504.0,20190104.0,NaN,NaN,NaN,NaN,NaN,월,2019-01


# Q) '월일' 컬럼을 만들어 보자!

In [30]:
# pandas.Series.dt.strftime 활용
raw['사용일자'].dt.strftime('%m-%d')


0         01-01
1         01-01
2         01-01
3         01-01
4         01-01
          ...  
107998      NaN
107999      NaN
108000      NaN
108001      NaN
108002      NaN
Name: 사용일자, Length: 108003, dtype: object

In [31]:
# 사용일자 컬럼에서 월일 값을 가져와 월일 컬럼 만들자

raw['월일'] = raw['사용일자'].dt.strftime('%m-%d')
raw.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,역이름,역지역,위도,경도,호선,요일,연월,월일
0,2019-01-01,2호선,을지로4가,3862.0,3728.0,20190104.0,NaN,NaN,NaN,NaN,NaN,월,2019-01,01-01
1,2019-01-01,3호선,을지로3가,8104.0,7554.0,20190104.0,NaN,NaN,NaN,NaN,NaN,월,2019-01,01-01
2,2019-01-01,2호선,을지로입구,22478.0,21330.0,20190104.0,NaN,NaN,NaN,NaN,NaN,월,2019-01,01-01
3,2019-01-01,2호선,시청,8381.0,6049.0,20190104.0,NaN,NaN,NaN,NaN,NaN,월,2019-01,01-01
4,2019-01-01,6호선,동묘앞,8045.0,8504.0,20190104.0,NaN,NaN,NaN,NaN,NaN,월,2019-01,01-01


# Q) 승자와 하자하는 모든 승객수를 구해 볼까요?

In [32]:
# 승하자총승객수 컬럼 생성 : 승차총승객수 + 하차총승객수
raw['승하자총승객수'] = raw['승차총승객수'] + raw['하차총승객수']

In [33]:
# 요일을 사용일자 다음에 나오도록 컬럼순서를 변경하겠습니다.

# 현재 컬럼명 확인
print(raw.columns)

raw = raw[['사용일자', '요일', '노선명', '역명', '승차총승객수', '하차총승객수', '승하자총승객수', '연월', '월일', '등록일자' ]]
raw.head()


Index(['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자', '역이름', '역지역', '위도',
       '경도', '호선', '요일', '연월', '월일', '승하자총승객수'],
      dtype='object')


,사용일자,요일,노선명,역명,승차총승객수,하차총승객수,승하자총승객수,연월,월일,등록일자
0,2019-01-01,월,2호선,을지로4가,3862.0,3728.0,7590.0,2019-01,01-01,20190104.0
1,2019-01-01,월,3호선,을지로3가,8104.0,7554.0,15658.0,2019-01,01-01,20190104.0
2,2019-01-01,월,2호선,을지로입구,22478.0,21330.0,43808.0,2019-01,01-01,20190104.0
3,2019-01-01,월,2호선,시청,8381.0,6049.0,14430.0,2019-01,01-01,20190104.0
4,2019-01-01,월,6호선,동묘앞,8045.0,8504.0,16549.0,2019-01,01-01,20190104.0


# 3. 정리한 데이터 저장하기

In [34]:
# 작업한 결과를 data 폴더에 저장하겠습니다.

fpath = './subway_raw.csv'
raw.to_csv(fpath, index=False, encoding='utf-8-sig')
